In [ ]:
# 주피터 노트북 환경설정
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

from IPython.display import Image

from IPython.core.display import display, HTML
# display(HTML("<style>.container { font-weight: bold !important; font-family:'Malgun Gothic' !important;}</style>"))
display(HTML("<style>.container { font-weight: bold !important;}</style>"))
display(HTML("<style>.container { width: 98% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
# plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

In [ ]:
from sklearn.ensemble import VotingClassifier, VotingRegressor

from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score, recall_score, precision_score, classification_report, r2_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.pipeline import Pipeline

# 앙상블(Ensemble) 

- Ensemble : 조화라는 사전적인 의미를 지님
- Ensemble Learning
    - 여러 개의 기본 모델을 활용하여 하나의 새로운 모델을 만들어내는 개념 
    - 기본모델(Base Model)을 weak learner, classifier, base 

###  보팅(Voting) 
    - 여러개의 분류기가 투표를 통헤 최종 예측 결과를 결정하는 방식 
    - 하드보팅 : 다수의 classifier 간 다수결의 최종 class 결정 
    - 소프트보팅 : 다수의 classfier 들의 class 확률을 평균하여 결정 
    - Voting Classifier¶ 활용 

    모델=[(키1,예측기1),(키2,예측기2)] , voting='soft/hard' )


# Voting Classification

## Voting

<img src='https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fb5UNZc%2FbtrhVZEJkJA%2F1bqKVdovvqY2NY050MWMSK%2Fimg.jpg'>

**위스콘신 유방암 데이터 로드**

- cancer.data
- cancer.feature_names
- cancer.target 

In [ ]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
cancer.keys()

In [ ]:
data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df.head(3)

In [ ]:
# 전체 갯수 확인
data_df.shape

In [ ]:
cancer.target[:30]

In [ ]:
# cancer.DESCR

### VotingClassifier로 개별모델은 로지스틱 회귀와 KNN을 보팅방식으로 결합하고 성능 비교

In [ ]:
# from sklearn.ensemble import VotingClassifier

model_lr = LogisticRegression() 
model_knn = KNeighborsClassifier(n_neighbors=8)

model_vo = VotingClassifier( estimators=[('LR',model_lr),('KNN',model_knn)] , voting='soft' )

X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    test_size=0.2 , random_state= 156)

model_vo.fit(X_train , y_train)
model_vo.score(X_train , y_train)
model_vo.score(X_test , y_test)


In [ ]:
classifiers = [model_lr, model_knn]
for clf in classifiers:
    clf .fit(X_train , y_train)
    print(clf, clf.score(X_test , y_test))
    print(classification_report(y_test, clf.predict(X_test)))

In [ ]:
# from sklearn.ensemble import VotingClassifier

model_lr = LogisticRegression() 
model_knn = KNeighborsClassifier(n_neighbors=8)

model_vo = VotingClassifier( estimators=[('LR',model_lr),('KNN',model_knn)] , voting='hard' )


X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, 
                                                    test_size=0.2 , random_state= 156)

model_vo.fit(X_train , y_train)
model_vo.score(X_train , y_train)
model_vo.score(X_test , y_test)

# Voting Regressor

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()

boston_df = pd.DataFrame(boston.data , columns = boston.feature_names)
 
boston_df['PRICE'] = boston.target

In [ ]:
y_target = boston_df['PRICE']
X_data = boston_df.drop(['PRICE'], axis=1, inplace=False)

X_train , X_test , y_train , y_test = train_test_split( X_data , y_target , \
                                                       test_size=0.3, random_state=156)

In [ ]:
# from sklearn.ensemble import VotingRegressor

model_linear = LinearRegression()
model_lasso = Lasso(alpha=10)
model_ridge = Ridge(alpha=10)

model_vo_r = VotingRegressor( estimators=[('LINEAR', model_linear),('LASSO', model_lasso), ('RIDGE', model_ridge)] )

model_vo_r.fit(X_train , y_train)
print(model_vo_r.score(X_train , y_train) , model_vo_r.score(X_test , y_test))

In [ ]:
model_vo_r.estimators_

In [ ]:
model_vo_r.named_estimators_

In [ ]:
model_vo_r.score(X_test , y_test), r2_score(y_test, model_vo_r.predict(X_test))

In [ ]:
# 퀴즈 